In [ ]:
import qctoolkit as qtk
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output


%matplotlib inline

clear_output()

## Load and process data

In [ ]:
inp_grp = qtk.load('A2_6003.pkl')

In [ ]:
print [len(inps) for inps in inp_grp]
print len(inp_grp)

In [ ]:
length = np.array([len(inps) for inps in inp_grp])
good_list = length > 200
inp_good = []
for i in np.arange(len(good_list))[good_list]:
    inps = inp_grp[i]
    E = np.array([o.Et for o in inps])
    if max(np.diff(E)) < 0.1:
        inp_good.append(inps)
        E = E - E[-1]
        R = [o.molecule.R[1,0] for o in inps]
        #plt.plot(np.diff(E))
        plt.plot(R,E)
print [inps[0].molecule.name for inps in inp_good]

In [ ]:
# for i in range(len(inp_grp)):
#     inps = inp_grp[i]
#     if len(inps) > 0:
#         E = np.array([o.Et for o in inps])
#         R = [o.molecule.R[1,0] for o in inps]
#         plt.figure()
#         plt.plot(R, E)
#         plt.title(inps[0].molecule.name + ' ind: %d' % i)

In [ ]:
def train_test_split(test='C1F1', validation='Li1O1'):
    good_test_list = {
        'H2': [0],
        'H1Li1': [1],
        'H1F1': [4],
        'Li1N1': [7],
        'Li1F1': [8],
        'N1F1': [13],
        'F2': [14],
        'Be1O1': [17],
        'H1Be1': [21], # cation
        'H1O1': [26], # anion
        'Li1C1': [30], # anion
        'Li1O1': [31, 32], # cat/anion
        'N1O1': [44], # anion
        'Be1F1': [45], # cation
        'C1F1': [47, 48], # cat/anion
        'O1F1': [49, 50], # cat/anion
    }
    
    test_inds = good_test_list[test]
    valid_inds = good_test_list[validation]
    train_inds = [i for i in range(len(inp_grp)) if i not in test_inds and i not in valid_inds]
    
    np.random.seed(42)
    
    inp_train = list(qtk.flatten([inp_grp[i] for i in train_inds]))
    inp_test = list(qtk.flatten([inp_grp[i] for i in test_inds]))
    inp_valid = list(qtk.flatten([inp_grp[i] for i in valid_inds]))
    
    for inp in [inp_train, inp_test, inp_valid]:
        np.random.shuffle(inp_train)
    
    return inp_train, inp_test, inp_valid
    
inp_train, inp_test, inp_valid = train_test_split()

## Tensorflow model

In [ ]:
import tensorflow as tf

In [ ]:
def get_output(I, Er, occ, nn, C):
    """take input tensor and approximated MO coefficients, C, to compute total energy"""
    K = I[1]
    Na = I[2]
    dm = (C * occ).dot(C.T)
    
    Ek = np.trace(dm.dot(K)) * 2
    Ev = np.trace(dm.dot(Na)) * 2
    Ej = np.trace(dm.dot(np.tensordot(dm, Er, axes=([0,1], [0,2])))) * 2
    Ex = -np.trace(dm.dot(np.tensordot(dm, Er, axes=([0,1], [0,1]))))
    E_tot = np.sum([Ek, Ev, Ej, Ex, nn])
    return E_tot

def hidden_layer(I, C_prd = None):
    
    if C_prd is None:
        C_prd = I[-2]
        
    S, K, V = I[0], I[1], I[2]
    
    Wc = tf.Variable(tf.truncated_normal(C_prd.shape), name='weights_C')
    Bc = tf.Variable(tf.zeros(C_prd.shape), name='biases_C')
    Ws = tf.Variable(tf.truncated_normal(C_prd.shape), name='weights_S')
    Bs = tf.Variable(tf.zeros(C_prd.shape), name='biases_S')
    Wk = tf.Variable(tf.truncated_normal(C_prd.shape), name='weights_K')
    Bk = tf.Variable(tf.zeros(C_prd.shape), name='biases_K')
    Wv = tf.Variable(tf.truncated_normal(C_prd.shape), name='weights_V')
    Bv = tf.Variable(tf.zeros(C_prd.shape), name='biases_V')
    
    C_new = tf.add(tf.matmul(C_prd, Wc), Bc)
    S_new = tf.add(tf.matmul(S, Ws), Bs)
    K_new = tf.add(tf.matmul(K, Wk), Bk)
    V_new = tf.add(tf.matmul(V, Wv), Bv)
    
    new_matrix = tf.add(tf.add(tf.add(C_new, S_new), K_new), V_new)
    
    return tf.nn.relu(new_matrix)

def normailization_layer(I, C_prd=None):
    
    if C_prd is None:
        C_prd = I[-2]
        
    W = tf.Variable(tf.truncated_normal(C_prd.shape), name='weights')
    B = tf.Variable(tf.zeros(C_prd.shape), name='biases')
    C_new = tf.add(tf.matmul(C_prd, W), B)
    
    C_sym = tf.matmul(tf.transpose(C_new), C_new)
    _, C_diag = tf.self_adjoint_eig(C_sym)
    
    return tf.matmul(I[-1], C_diag)

def output_layer(I, Er, occ, nn, C_prd):
    K, Na = I[1], I[2]
    
    C_occ = tf.multiply(occ, C_prd)
    #C_occ = tf.matmul(tf.expand_dims(0, occ), C)
    dm = tf.matmul(C_occ, tf.transpose(C_prd))
    
    J_kernel = tf.tensordot(dm, Er, axes=([0,1], [0,2]))
    X_kernel = tf.tensordot(dm, Er, axes=([0,1], [0,1]))
    
    Ek = tf.trace(tf.matmul(dm, K)) * 2
    Ev = tf.trace(tf.matmul(dm, Na)) * 2
    Ej = tf.trace(tf.matmul(dm, J_kernel)) * 2
    Ex = -tf.trace(tf.matmul(dm, X_kernel))
    
    E_total = tf.add(Ek, Ev)
    E_total = tf.add(E_total, Ej)
    E_total = tf.add(E_total, Ex)
    E_total = tf.add(E_total, nn)
    
    return E_total

def tf_horton_interface(mol):
    new_mol = qtk.QMInp(mol.molecule, program='horton', basis_set=basis)
    dm, C0_np, S_np, K_np, Na_np, Er_np = new_mol.matrices()
    occ_np = new_mol.occ
    D, U = np.linalg.eig(new_mol.olp)
    X_np = U / np.sqrt(D)
    I_np = np.stack([S_np.real, K_np.real, Na_np.real, C0_np.real, X_np.real])
    nn_np = new_mol.ht_external['nn']

    C_np = mol.mov
    y_np = get_output(I_np, Er_np, occ_np, nn_np, C_np)
    
    #train_dict = {I:I_np, Er:Er_np, occ:occ_np, nn:nn_np, y:y_np}
    
    #return train_dict, C_np
    return I_np, Er_np, occ_np, nn_np, C_np, y_np

## Check results

In [ ]:
import horton as ht
basis = ht.GOBasisFamily('basis', filename='basis/sto2g/H_Ne_uncontracted.nwchem')

In [ ]:
with tf.Session() as sess:

    def check_E_tf(mol):
        I, Er, occ, nn, C, y = tf_horton_interface(mol)
    
        E_tf = sess.run(output_layer(I, Er, occ, nn, C))
        return E_tf - mol.Et

    err_E = [check_E_tf(inp) for inp in inp_test]
    print err_E
    print np.max(np.abs(err_E))

In [ ]:
def check_E(mol):
    I, Er, occ, nn, C, y = tf_horton_interface(mol)
    S, K, V, C0, X = I
    Et = mol.Et
    
    dm = mol.dm(C)
    
    J_kernel = np.tensordot(dm, Er, axes=([0,1], [0,2]))
    X_kernel = np.tensordot(dm, Er, axes=([0,1], [0,1]))
    
    Ek = np.trace(dm.dot(K)) * 2
    Ev = np.trace(dm.dot(V)) * 2
    Ej = np.trace(dm.dot(J_kernel)) * 2
    Ex = -np.trace(dm.dot(X_kernel))
    
    E = Ek + Ev + Ej + Ex + nn
    
    return Et - E

print check_E(inp_test[0])

for inp in inp_test:
    print inp.molecule.name, 
    print check_E(inp)
#print [check_E_tf(inp) for inp in inp_test]

## Build and train tensorflow model

In [ ]:
tf.reset_default_graph()
log = open('A2_run.log', 'w')

def get_shape(tensor):
    shape = list(tensor.shape)
    #shape.insert(0, None)
    return shape

mol = inp_train[0]
new_mol = qtk.QMInp(mol.molecule, program='horton', basis_set=basis)
dm, C0_np, S_np, K_np, Na_np, Er_test = new_mol.matrices()
occ_test = new_mol.occ
D, U = np.linalg.eig(new_mol.olp)
X_np = U / np.sqrt(D)
I_test = np.stack([S_np, K_np, Na_np, C0_np, X_np])
nn_test = new_mol.ht_external['nn']


# input tensors
I = tf.placeholder(tf.float32, shape=get_shape(I_test), name='I')
Er = tf.placeholder(tf.float32, shape=get_shape(Er_test), name='Er')
occ = tf.placeholder(tf.float32, shape=get_shape(occ_test), name='occ')
nn = tf.placeholder(tf.float32, shape=get_shape(nn_test), name='nn')

# training tensors
C_ref = tf.placeholder(tf.float32, shape=get_shape(C0_np), name='C_ref')
y = tf.placeholder(tf.float32, name='y')

# output tensor
C_prd = hidden_layer(I)
for _ in range(2):
    C_prd = hidden_layer(I, C_prd)
C_prd = tf.nn.dropout(C_prd, 0.8)
for _ in range(2):
    C_prd = hidden_layer(I, C_prd)
#for _ in range(3):
#C_prd = tf.nn.dropout(C_prd, 0.8)
C_prd = normailization_layer(I, C_prd)
C_prd = normailization_layer(I, C_prd)
y_prd = output_layer(I, Er, occ, nn, C_prd)

err = tf.pow(y - y_prd, 2)

optimizer = tf.train.GradientDescentOptimizer(0.02).minimize(err)
#optimizer = tf.train.AdamOptimizer(0.01).minimize(err) # default learning rate 0.001

err_list = []
err_list_valid = []
itr_list = []
E_prd_list = []
C_prd_list = []


save_path = "A2-6000_E-5.ckpt"
saver = tf.train.Saver()

sess = tf.Session()
#with tf.Session() as sess:
    
try:
    try:
        saver.restore(sess, save_path)
        msg = 'model loaded, continue optimizing'
    except:
        sess.run(tf.global_variables_initializer())
        msg = 'no model found, start from scratch'
    print msg
    log.write(msg)
    log.write("\n")
    log.flush()
    #saver.restore(sess, "/path/to/foo.ckpt")

    valid_err = 1
    opt_itr = 0
    #while valid_err > 0.00001:
    for _ in range(20):
        msg = "%2d optimization run" % (opt_itr+1) 
        print msg
        log.write(msg)
        log.write("\n")
        log.flush()
        itr = 0
        for mol in inp_train:
            
            I_train, Er_train, occ_train, nn_train, C_train, y_train = tf_horton_interface(mol)
            train_dict = {I:I_train, Er:Er_train, occ:occ_train, nn:nn_train, y:y_train}

            sess.run(optimizer, feed_dict=train_dict)

            train_err = sess.run(err, feed_dict=train_dict)

            err_list.append(train_err)

            if itr % 5 == 0:
                valid_errs = []
                for mol_v in inp_valid:
                    I_v, Er_v, occ_v, nn_v, C_v, y_v = tf_horton_interface(mol_v)
                    valid_dict = {I:I_v, Er:Er_v, occ:occ_v, nn:nn_v, y:y_v}
                    valid_errs.append(sess.run(err, feed_dict=valid_dict))
                valid_err = np.array(valid_errs).mean()
                err_list_valid.append(valid_err)
                itr_list.append(itr)
                msg = "itr: %4d, training error: %f, test error %f" % (itr, train_err, valid_err)
                print msg
                log.write(msg)
                log.write("\n")
                log.flush()
            itr += 1
        opt_itr += 1
    
    for mol_t in inp_test:
        
        I_t, Er_t, occ_t, nn_t, C_t, y_t = tf_horton_interface(mol_t)
        test_dict = {I:I_t, Er:Er_t, occ:occ_t, nn:nn_t, y:y_t}

        E_prd_list.append(sess.run(y_prd, feed_dict=test_dict))
        C_prd_list.append(sess.run(C_prd, feed_dict=test_dict))
        
    save_path_out = saver.save(sess, save_path)
    print("Model saved in file: %s" % save_path_out)
    
    log.close()
    
except KeyboardInterrupt:
    print "keyboard interrupt"
    save_path_out = saver.save(sess, save_path)
    print("Model saved in file: %s" % save_path_out)
    session.close()
    log.close